In [23]:

import numpy as np

import pandas as pd

x_train = pd.read_csv('train_features.csv')
x_test = pd.read_csv('test_features.csv')
y_train = pd.read_csv('train_labels.csv')
y_test = pd.read_csv('test_labels.csv')
y_train=y_train.labels
y_test=y_test.labels


In [25]:
print(x_train.dna)
print(y_train)
print(y_train[0])

0        AACATTATACTTTATTTTCGGAGCATGATCAGGAATAGTAGGAACT...
1        TACACTATACTTCATTTTTGGTGCTTGAGCAGGAATGCTAGGAACA...
2        ----------------------------------------------...
3        AACATTATATTTTATTTTTGGTGCATGAGCTGGAATAGTAGGAACT...
4        AACTTTATATTTTATTTTTGGAGCTTGAGCTGGAATAGTTGGAACA...
                               ...                        
12901    TACTCTGTATTTTCTATTTGGAGTATGATCAGGAATAGTAGGAACA...
12902    AACATTATATTTTATCTTTGGGGCCTGATCAGGAATAGTAGGAACT...
12903    AACCTTATATTTCCTATTCGGAGCATGAGCCGGAATATTAGGAACA...
12904    ----------------------------------------------...
12905    AACATTATATTTTATTTTTGGGGCTTGAGCTGGAATAGTTGGAACT...
Name: dna, Length: 12906, dtype: object
0          33
1         634
2        1175
3          32
4         468
         ... 
12901     400
12902     171
12903     329
12904    1148
12905     406
Name: labels, Length: 12906, dtype: int64
33


In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Converting base pairs, ACTG, into numbers
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(x_train.dna)
sequence_of_int_train = tokenizer.texts_to_sequences(x_train.dna)
sequence_of_int_test = tokenizer.texts_to_sequences(x_test.dna)

In [10]:
Ntrain=len(sequence_of_int_train)
Ntest=len(sequence_of_int_test)

In [11]:

sl = 658
Xtrain=np.zeros((Ntrain,sl,5))
Xtest=np.zeros((Ntest,sl,5))
for i in range(Ntrain):
    Nt=len(sequence_of_int_train[i])

    for j in range(sl):
        if(len(sequence_of_int_train[i])>j):
            k=sequence_of_int_train[i][j]-1
            if(k>4):
                k=4
            Xtrain[i][j][k]=1.0
            
for i in range(Ntest):
    Nt=len(sequence_of_int_test[i])

    for j in range(sl):
        if(len(sequence_of_int_test[i])>j):
            k=sequence_of_int_test[i][j]-1
            if(k>4):
                k=4
            Xtest[i][j][k]=1.0

In [17]:
print(y_train)

          id  labels
0          1      33
1          2     634
2          3    1175
3          4      32
4          5     468
...      ...     ...
12901  12902     400
12902  12903     171
12903  12904     329
12904  12905    1148
12905  12906     406

[12906 rows x 2 columns]


In [19]:
# Expanding the training set shape for CNN 
Xtrain=np.expand_dims(Xtrain, axis=3)
Xtest=np.expand_dims(Xtest, axis=3)
print(Xtrain.shape)

(12906, 658, 5, 1)


In [46]:
# for now just set test -1s to pos int outside of range to check error locally
for k in range(len(y_test)):
    if y_test[k]==-1:
        y_test[k]=1217

In [30]:
print(np.max(y_train))

1213


### Classic CNN Model

In [20]:
from tensorflow.keras import datasets, layers, models, optimizers, callbacks

In [47]:
# CNN model architecture
model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation='relu', input_shape=(sl, 5,1),padding="SAME"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((3,1)))
model.add(layers.Conv2D(32, (3,3), activation='relu',padding="SAME"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((3,1)))
model.add(layers.Conv2D(16, (3,3), activation='relu',padding="SAME"))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((3,1)))
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(500, activation='tanh'))
model.add(layers.Dense(1218))

In [48]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 658, 5, 64)        640       
_________________________________________________________________
batch_normalization_20 (Batc (None, 658, 5, 64)        256       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 219, 5, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 219, 5, 32)        18464     
_________________________________________________________________
batch_normalization_21 (Batc (None, 219, 5, 32)        128       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 73, 5, 32)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 73, 5, 16)        

In [49]:
# Step-decay learning rate scheduler
def step_decay(epoch):
   initial_lrate = 0.001
   drop = 0.5
   epochs_drop = 2.0
   lrate = initial_lrate * np.power(drop, np.floor((1+epoch)/epochs_drop))
   return lrate

class LossHistory(callbacks.Callback):
    def on_train_begin(self, logs={}):
       self.losses = []
       self.lr = []
 
    def on_epoch_end(self, batch, logs={}):
       self.losses.append(logs.get('loss'))
       self.lr.append(step_decay(len(self.losses)))
        
loss_history = LossHistory()
lrate = callbacks.LearningRateScheduler(step_decay)
callbacks_list = [loss_history, lrate]

In [50]:
from tensorflow.keras.metrics import top_k_categorical_accuracy
import tensorflow as tf
#opt = tf.keras.optimizers.SGD(momentum=0.9, nesterov=True)
opt = tf.keras.optimizers.Adam()
model.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy','top_k_categorical_accuracy'])

# Validation time
history = model.fit(Xtrain, y_train, epochs=5, batch_size = 32, validation_data=(Xtest, y_test), callbacks=callbacks_list, verbose=1)



Epoch 1/5
404/404 [==============================] - 43s 106ms/step - loss: 1.8177 - accuracy: 0.7784 - top_k_categorical_accuracy: 2.3245e-04 - val_loss: 4.9266 - val_accuracy: 0.1539 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 2/5
404/404 [==============================] - 43s 106ms/step - loss: 0.0874 - accuracy: 0.9888 - top_k_categorical_accuracy: 0.0000e+00 - val_loss: 0.3310 - val_accuracy: 0.9558 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 3/5
404/404 [==============================] - 44s 108ms/step - loss: 0.0291 - accuracy: 0.9959 - top_k_categorical_accuracy: 0.0000e+00 - val_loss: 0.3015 - val_accuracy: 0.9588 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 4/5
404/404 [==============================] - 45s 112ms/step - loss: 0.0170 - accuracy: 0.9970 - top_k_categorical_accuracy: 0.0000e+00 - val_loss: 0.2929 - val_accuracy: 0.9598 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 5/5
404/404 [==============================] - 45s 111ms/step - loss: 0.01

In [53]:

X_new_train=Xtrain.reshape((Xtrain.shape[0], 658*5))

In [54]:
# make sampling layer
original_dim=658*5
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.losses import binary_crossentropy
import tensorflow.keras.backend as K
intermediate_dim = 256

# can change to 50 or 2
latent_dim = 50
epoch_num=30
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Define encoder model.
original_inputs = tf.keras.Input(shape=(original_dim,), name="encoder_input")
x = layers.Dense(intermediate_dim, activation="relu")(original_inputs)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()((z_mean, z_log_var))
encoder = tf.keras.Model(inputs=original_inputs, outputs=z, name="encoder")

# Define decoder model.
latent_inputs = tf.keras.Input(shape=(latent_dim,), name="z_sampling")
x = layers.Dense(intermediate_dim, activation="relu")(latent_inputs)
outputs = layers.Dense(original_dim, activation="sigmoid")(x)
decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name="decoder")

# Define VAE model.
outputs = decoder(z)
vae = tf.keras.Model(inputs=original_inputs, outputs=outputs, name="vae")

# Add KL divergence regularization loss.
kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
reconstruction_loss = binary_crossentropy(original_inputs,outputs)
reconstruction_loss *= original_dim
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)


# Train.
optimizer = tf.keras.optimizers.Adam(learning_rate=.001)
vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(X_new_train, X_new_train, epochs=epoch_num, batch_size=128)

Epoch 1/30
101/101 [==============================] - 2s 14ms/step - loss: 682.4698
Epoch 2/30
101/101 [==============================] - 1s 15ms/step - loss: 427.0390
Epoch 3/30
101/101 [==============================] - 1s 14ms/step - loss: 376.2299
Epoch 4/30
101/101 [==============================] - 1s 14ms/step - loss: 344.0372
Epoch 5/30
101/101 [==============================] - 1s 14ms/step - loss: 319.8466
Epoch 6/30
101/101 [==============================] - 1s 14ms/step - loss: 299.6163
Epoch 7/30
101/101 [==============================] - 1s 14ms/step - loss: 282.7689
Epoch 8/30
101/101 [==============================] - 1s 15ms/step - loss: 268.0807
Epoch 9/30
101/101 [==============================] - 1s 15ms/step - loss: 255.2028
Epoch 10/30
101/101 [==============================] - 2s 15ms/step - loss: 243.4993
Epoch 11/30
101/101 [==============================] - 1s 15ms/step - loss: 233.1757
Epoch 12/30
101/101 [==============================] - 2s 15ms/step - loss

In [55]:
x_train_encoded = encoder.predict(X_new_train, batch_size=36)
X_test_new=Xtest.reshape((Xtest.shape[0], 658*5))

x_test_encoded = encoder.predict(X_test_new, batch_size=36)

In [ ]:
# now train simple MLP classifier using VAE embeddings

In [59]:
model2 = models.Sequential()
model2.add(layers.Dense(300))
model2.add(layers.Dense(1218))

model2.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy','top_k_categorical_accuracy'])

history = model2.fit(x_train_encoded, y_train, epochs=20, batch_size = 32, validation_data=(x_test_encoded, y_test), callbacks=callbacks_list, verbose=1)

Epoch 1/20
404/404 [==============================] - 2s 4ms/step - loss: 1.8234 - accuracy: 0.7555 - top_k_categorical_accuracy: 0.0000e+00 - val_loss: 0.6804 - val_accuracy: 0.9065 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 2/20
404/404 [==============================] - 1s 3ms/step - loss: 0.2390 - accuracy: 0.9549 - top_k_categorical_accuracy: 0.0000e+00 - val_loss: 0.5600 - val_accuracy: 0.9298 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 3/20
404/404 [==============================] - 1s 3ms/step - loss: 0.1573 - accuracy: 0.9662 - top_k_categorical_accuracy: 0.0000e+00 - val_loss: 0.5695 - val_accuracy: 0.9313 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 4/20
404/404 [==============================] - 1s 3ms/step - loss: 0.1042 - accuracy: 0.9764 - top_k_categorical_accuracy: 0.0000e+00 - val_loss: 0.5499 - val_accuracy: 0.9370 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 5/20
404/404 [==============================] - 1s 3ms/step - loss: 0.0861 - accur